# Brain Tumor Detection CNN Model

---

This is a ML model created by Keshav Ghai (An aspiring AI/ML dev).

This is a **Convolutional Neural Network (CNN)** which detects and classifies brain tumors from MRI scan images. Unlike binary classification models, this is a **multi-class classifier** that can distinguish between different types of brain tumors. The training script **"trainer.py"** takes MRI image data, preprocesses it, trains a CNN with 3 convolutional layers, and generates visualizations for model performance analysis.

## What makes this different?

This model performs **multi-class classification** (typically 4 classes: no tumor, glioma, meningioma, pituitary). CNNs are ideal for medical imaging because they automatically learn to detect complex patterns in images—from simple edges to complex anatomical structures. The model helps radiologists by providing a second opinion on brain tumor classification from MRI scans.

## Imports:-
---

In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

## 0. Dataset Normalization (Data Preprocessing)
---

Before training the model, the MRI dataset must be normalized. It is done through these steps:

**Why normalize?** Medical images come from different scanners with different pixel value ranges. Normalization ensures consistency so the model can learn general patterns rather than scanner-specific artifacts.

### Normalization Steps:
1. **Read MRI images** from raw dataset folders
2. **Convert to RGB** - standardize color format for consistency
3. **Resize to 256×256** - ensures all images have the same dimensions for the model input
4. **Organize into Train/Val/Test splits** - separate directories for training, validation, and testing
5. **Save normalized images** to `normalized_dataset` with proper folder structure maintaining class labels

The dataset structure should be:
```
normalized_dataset/
├── Train/
│   ├── glioma/
│   ├── meningioma/
│   ├── notumor/
│   └── pituitary/
└── Val/
    ├── glioma/
    ├── meningioma/
    ├── notumor/
    └── pituitary/
```

In [ ]:
import os
import cv2

# Paths for raw and normalized datasets
RAW_DATASET = "./tensorflow/tumor_detection/dataset"
OUTPUT_DATASET = "./tensorflow/tumor_detection/normalized_dataset"

TARGET_SIZE = (256, 256)

def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def process_split(split):
    input_dir = os.path.join(RAW_DATASET, split)
    output_dir = os.path.join(OUTPUT_DATASET, split)
    ensure_dir(output_dir)

    classes = os.listdir(input_dir)

    for cls in classes:
        cls_input = os.path.join(input_dir, cls)
        cls_output = os.path.join(output_dir, cls)
        ensure_dir(cls_output)

        for img_name in os.listdir(cls_input):
            img_path = os.path.join(cls_input, img_name)
            img = cv2.imread(img_path)

            if img is None:
                continue

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, TARGET_SIZE)

            out_path = os.path.join(cls_output, os.path.splitext(img_name)[0] + ".png")
            cv2.imwrite(out_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

    print(f"{split} split processed.")

def main():
    process_split("Train")
    process_split("Val")
    print("Normalization complete.")

if __name__ == "__main__":
    main()


## 1. Define Paths and Constants
---

> These variables point to the normalized dataset directories and where to save the trained model and results.

In [ ]:
# Paths to normalized dataset
DATASET_DIR = "./tensorflow/tumor_detection/normalized_dataset"

# Model output paths
MODEL_SAVE_PATH = "./Models/brain_tumor_model.keras"
CLASS_INDICES_PATH = "./tensorflow/tumor_detection/class_indices.json"

# Directory for saving plots
PLOT_DIR = "./tensorflow/tumor_detection"

# Image and training settings
IMG_SIZE = (256, 256)  # Height x Width of MRI images
BATCH_SIZE = 32        # Number of images to process together

print(f"Dataset loaded from: {DATASET_DIR}")
print(f"Model will be saved to: {MODEL_SAVE_PATH}")

## 2. Load Training and Validation Datasets
---

> TensorFlow's `image_dataset_from_directory` automatically loads images and infers class labels from folder names (e.g., "glioma", "meningioma", "notumor", "pituitary").

In [ ]:
# Load training data
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATASET_DIR, "Train"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int"  # Integer labels for sparse categorical crossentropy
)

# Load validation data
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATASET_DIR, "Val"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int"
)

# Get class names from the dataset
class_names = train_ds.class_names
num_classes = len(class_names)

print(f"Classes detected: {class_names}")
print(f"Number of classes: {num_classes}")

# Save class index mapping for later use during inference
with open(CLASS_INDICES_PATH, "w") as f:
    json.dump({cls: i for i, cls in enumerate(class_names)}, f)

print(f"Class mapping saved to: {CLASS_INDICES_PATH}")

# Prefetch: Load next batch while current batch is being processed (optimization)
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

print("\nDataset loaded successfully!")

## 3. Understanding the Multi-Class CNN Architecture
---

> This CNN learns to distinguish between 4 different tumor classes. It uses hierarchical feature extraction: early layers detect simple patterns (edges, textures in MRI), middle layers combine them (tissue structures), and later layers recognize high-level features (tumor characteristics).

### Architecture Components:

**Rescaling Layer**: Normalizes pixel values from [0, 255] to [0, 1] for better neural network learning.

**Conv2D Layers**: Extract features from MRI images using filters. `Conv2D(32, (3,3))` means 32 filters of size 3×3 pixels.

**MaxPooling2D**: Reduces spatial dimensions and computational cost while preserving important features.

**Flatten**: Converts 2D feature maps into a 1D vector for dense layers.

**Dense Layers**: Learn high-level patterns and relationships between extracted features.

**Dropout(0.3)**: Randomly ignores 30% of neurons to prevent overfitting.

**Output Layer**: Softmax activation with N neurons (one for each tumor class) for multi-class classification.

In [ ]:
# Build the multi-class CNN model
model = tf.keras.Sequential([
    # Normalize pixel values
    tf.keras.layers.Rescaling(1./255, input_shape=(*IMG_SIZE, 3)),

    # First convolutional block: Extract low-level features
    tf.keras.layers.Conv2D(32, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(),

    # Second convolutional block: Combine and refine features
    tf.keras.layers.Conv2D(16, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(),

    # Third convolutional block: Learn complex patterns
    tf.keras.layers.Conv2D(16, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(),

    # Flatten and fully connected layers
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    
    # Multi-class output layer (softmax for probability distribution)
    tf.keras.layers.Dense(num_classes, activation="softmax")
])

# Compile the model
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",  # Loss for integer-encoded multi-class labels
    metrics=["accuracy"]
)

# Display model architecture
model.summary()

## 4. Training the Model
---

> The model learns by:
1. Making predictions on training MRI images
2. Computing loss between predictions and actual tumor classes
3. Backpropagating error and adjusting weights
4. Repeating for 12 epochs (passes through entire training dataset)

Each epoch processes all batches of 32 images and updates model weights.

In [ ]:
# Train the model
history = model.fit(
    train_ds,                    # Training data with labels
    validation_data=val_ds,      # Validation data for monitoring
    epochs=12                    # Number of complete passes through dataset
)

# Save the trained model
model.save(MODEL_SAVE_PATH)
print(f"\nModel saved at: {MODEL_SAVE_PATH}")

## 5. Evaluate Model Performance
---

> We test the model on both training and validation data:
- **Training Accuracy**: How well it learned from training data
- **Validation Accuracy**: How well it generalizes to unseen data (more important)

Large gap between training and validation accuracy indicates overfitting.

In [ ]:
# Evaluate on training set
train_loss, train_acc = model.evaluate(train_ds)

# Evaluate on validation set
val_loss, val_acc = model.evaluate(val_ds)

print(f"\n=== Model Performance ===")
print(f"Training Accuracy:   {train_acc * 100:.2f}%")
print(f"Validation Accuracy: {val_acc * 100:.2f}%")
print(f"\nTraining Loss:   {train_loss:.4f}")
print(f"Validation Loss: {val_loss:.4f}")

## 6. Visualizations
---

> Graphs show model learning progress over epochs and prediction accuracy across all tumor classes.

### a. Loss Over Epochs:-

Lower loss is better. Both curves should decrease smoothly. If validation loss increases sharply, the model is overfitting.

In [ ]:
# Plot loss graph
plt.figure(figsize=(6,4))
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.title("Loss Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(PLOT_DIR, "loss_graph.png"))
plt.close()

print("Loss graph saved")

### b. Accuracy Over Epochs:-

Higher accuracy is better. A steeply rising curve indicates the model is learning well.

In [ ]:
# Plot accuracy graph
plt.figure(figsize=(6,4))
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Val Accuracy")
plt.title("Accuracy Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(PLOT_DIR, "accuracy_graph.png"))
plt.close()

print("Accuracy graph saved")

### c. Confusion Matrix (Multi-Class):-

Shows how the model classifies each tumor type:
- **Diagonal values**: Correct predictions for each class
- **Off-diagonal values**: Misclassifications (which classes get confused with each other)

Ideally, high diagonal values and low off-diagonal values indicate good performance.

In [ ]:
# Generate confusion matrix on validation set
y_true = []
y_pred = []

# Predict on all validation batches
for images, labels in val_ds:
    preds = model.predict(images, verbose=0)
    preds = np.argmax(preds, axis=1)  # Get class with highest probability

    y_true.extend(labels.numpy())
    y_pred.extend(preds)

# Create confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot as heatmap
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", 
            xticklabels=class_names, 
            yticklabels=class_names,
            cmap="Blues")
plt.title("Confusion Matrix (Validation Set)")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.savefig(os.path.join(PLOT_DIR, "confusion_matrix.png"), bbox_inches='tight')
plt.close()

print("Confusion matrix saved.")

## 7. Interactive Testing Mode
---

> Test the trained model on any MRI image. The model will:
1. Read and preprocess the image (resize to 256×256, normalize)
2. Make a prediction using the trained CNN
3. Output the tumor class and confidence score
4. Provide clinical interpretation (Tumor Present: YES/NO)

The confidence score (0-100%) indicates how certain the model is about its prediction.

In [ ]:
print("\nInteractive Testing Mode")
print(f"Classes available: {class_names}")
print("Type an image path to test, or 'quit' to exit.\n")

while True:
    user_input = input("Image path: ").strip()

    if user_input.lower() == "quit":
        print("Exiting interactive mode.")
        break

    if not os.path.exists(user_input):
        print("File not found. Try again.\n")
        continue

    try:
        # Load and preprocess image
        img = tf.keras.preprocessing.image.load_img(user_input, target_size=IMG_SIZE)
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = np.expand_dims(img, axis=0)  # Add batch dimension
        img = img / 255.0  # Normalize

        # Make prediction
        preds = model.predict(img, verbose=0)
        class_id = np.argmax(preds[0])
        class_name = class_names[class_id]
        confidence = float(np.max(preds[0]) * 100)

        print(f"\nPredicted Class: {class_name}")
        print(f"Confidence: {confidence:.2f}%")

        # Clinical interpretation
        if class_name == "notumor":
            print("Clinical: Tumor Present - NO")
        else:
            print(f"Clinical: Tumor Present - YES ({class_name.upper()})")
        
        # Show all class probabilities
        print("\nAll Class Probabilities:")
        for i, cls in enumerate(class_names):
            print(f"  {cls}: {preds[0][i]*100:.2f}%")
        print()

    except Exception as e:
        print(f"Error processing image: {e}\n")